In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
epsilon_0 = 8.854e-12  # Vacuum permittivity
R = 0.5  # Radius of disk (m)
sigma = 1e-3  # Surface charge density (C/m^2)
z0 = 1.0  # Observation height (m)

# Compute the electric field at each (x, y, z)
def efield_disk(x, y, z, R, sigma):
    r = np.sqrt(x**2 + y**2)  # Radial distance from the z-axis
    
    # Compute E_z component
    E_z = (sigma / (2 * epsilon_0)) * (1 - z / np.sqrt(R**2 + z**2))

    # Compute the radial E_r component
    if r == 0:
        return np.array([0, 0, E_z])  # No radial component at r=0

    E_r = (sigma / (2 * epsilon_0)) * (R**2 / (z * np.sqrt(R**2 + z**2)))

    # Convert radial component into Cartesian components
    E_x = E_r * (x / r)
    E_y = E_r * (y / r)

    return np.array([E_x, E_y, E_z])

# Grid for visualization
x_vals = np.linspace(-2.0, 2.0, 20)
y_vals = np.linspace(-2.0, 2.0, 20)
X, Y = np.meshgrid(x_vals, y_vals)

Ex_vals = np.zeros_like(X)
Ey_vals = np.zeros_like(Y)
Ez_vals = np.zeros_like(X)

# Compute E-field at each grid point
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        E = efield_disk(X[i, j], Y[i, j], z0, R, sigma)
        Ex_vals[i, j], Ey_vals[i, j], Ez_vals[i, j] = E

# Plot the field without opacity scaling
plt.figure(figsize=(6, 6))
quiver = plt.quiver(X, Y, Ex_vals, Ey_vals, color='b', scale=1e9)  # Adjusted scale
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.title(f"Electric Field of a Charged Disk at z = {z0} m")
plt.show()


## Point Charge Ring Estimation

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Constants
epsilon_0 = 8.854e-12  # Vacuum permittivity
R = 1.0  # Radius of ring (m)
Q = 1e-6  # Total charge (C)
z0 = 3.0  # Observation height (m)
q = Q / 4  # Each point charge carries Q/4

# Observation point (on the z-axis)
obs_point = np.array([0, 0, z0])

# Define four point charges on the ring (at 0, 90, 180, 270 degrees)
angles = np.radians([0, 90, 180, 270])
charge_points = np.array([[R * np.cos(theta), R * np.sin(theta), 0] for theta in angles])

# Compute the field contribution from each charge
E_total = np.array([0.0, 0.0, 0.0])

for point in charge_points:
    r_vec = obs_point - point  # Vector from charge to observation point
    r_mag = np.linalg.norm(r_vec)  # Magnitude of r vector
    dE_mag = (1 / (4 * np.pi * epsilon_0)) * (q / r_mag**3)  # Coulomb's Law
    dE_vec = dE_mag * r_vec  # Field contribution
    E_total += dE_vec  # Sum up contributions

# Round the E-field components to the nearest 10^-6
E_total = np.round(E_total, 2)

# Print results
print(f"Total Electric Field at z = {z0} m: E_x = {E_total[0]:.2f} V/m , E_y = {E_total[1]:.2f} V/m, E_z = {E_total[2]:.2f}V/m")

# Visualization
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)

# Plot the ring
circle = plt.Circle((0, 0), R, color='gray', fill=False, linestyle='dashed')
ax.add_patch(circle)

# Plot the four charge points
plt.scatter(charge_points[:, 0], charge_points[:, 1], color='red', label="Point Charges")

# Plot the observation point
plt.scatter(0, 0, color='blue', marker='x', s=100, label="Observation Point (on z-axis)")

# Formatting
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.legend()
plt.title("Approximation of Ring by 4 Point Charges and E-field at z-axis")
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
plt.grid()
plt.show()


# Field from Single charge

In [ ]:
import numpy as np

def electric_field(charge, charge_position, field_point):
    """
    Calculate the electric field vector at a point due to a point charge.
    
    Parameters:
    -----------
    charge : float
        The charge in Coulombs
    charge_position : array-like
        The position of the charge as [x, y, z] coordinates
    field_point : array-like
        The position where we want to calculate the field as [x, y, z] coordinates
        
    Returns:
    --------
    E_field : numpy.ndarray
        The electric field vector [Ex, Ey, Ez] in N/C (Newtons per Coulomb)
    """
    # Convert inputs to numpy arrays
    charge_position = np.array(charge_position)
    field_point = np.array(field_point)
    
    # Calculate displacement vector from charge to field point
    r_vector = field_point - charge_position
    
    # Calculate distance
    r = np.linalg.norm(r_vector)
    
    # Check for singularity (field point at charge position)
    if r < 1e-10:
        print("Warning: Field point too close to charge. Returning zero field.")
        return np.zeros(3)
    
    # Calculate unit vector
    r_hat = r_vector / r
    
    # Calculate electric field magnitude
    # E = kq/r², where k is Coulomb's constant
    k = 8.9875e9  # Coulomb's constant (N·m²/C²)
    E_magnitude = k * charge / (r**2)
    
    # Calculate electric field vector
    E_field = E_magnitude * r_hat
    
    return E_field


if __name__ == "__main__":
    # Example usage
    charge = 1e-9  # 1 nC (nanocoulomb)
    charge_position = [0, 0, 0]  # At the origin
    
# Calculate field at a specific point
field_point = [1, 1, 1]  # 1 meter in each direction
E = electric_field(charge, charge_position, field_point)
print(f"Electric field at {field_point} is [{E[0]:.3e}, {E[1]:.3e}, {E[2]:.3e}] N/C")
print(f"Field magnitude: {np.linalg.norm(E):.4e} N/C")

# Field from multiple charges

In [ ]:
import numpy as np

def electric_field(charge, charge_position, field_point):
    """
    Calculate the electric field vector at a point due to a point charge.
    
    Parameters:
    -----------
    charge : float
        The charge in Coulombs
    charge_position : array-like
        The position of the charge as [x, y, z] coordinates
    field_point : array-like
        The position where we want to calculate the field as [x, y, z] coordinates
        
    Returns:
    --------
    E_field : numpy.ndarray
        The electric field vector [Ex, Ey, Ez] in N/C (Newtons per Coulomb)
    """
    # Convert inputs to numpy arrays
    charge_position = np.array(charge_position)
    field_point = np.array(field_point)
    
    # Calculate displacement vector from charge to field point
    r_vector = field_point - charge_position
    
    # Calculate distance
    r = np.linalg.norm(r_vector)
    
    # Check for singularity (field point at charge position)
    if r < 1e-10:
        print("Warning: Field point too close to charge. Returning zero field.")
        return np.zeros(3)
    
    # Calculate unit vector
    r_hat = r_vector / r
    
    # Calculate electric field magnitude
    # E = kq/r², where k is Coulomb's constant
    k = 8.9875e9  # Coulomb's constant (N·m²/C²)
    E_magnitude = k * charge / (r**2)
    
    # Calculate electric field vector
    E_field = E_magnitude * r_hat
    
    return E_field

def electric_field_multiple_charges(charges, charge_positions, field_point):
    """
    Calculate the total electric field vector at a point due to multiple point charges.
    Uses the principle of superposition.
    
    Parameters:
    -----------
    charges : list or numpy.ndarray
        List of charge values in Coulombs
    charge_positions : list of lists or numpy.ndarray
        List of positions for each charge as [x, y, z] coordinates
    field_point : list or numpy.ndarray
        The position where we want to calculate the field as [x, y, z] coordinates
        
    Returns:
    --------
    total_E_field : numpy.ndarray
        The total electric field vector [Ex, Ey, Ez] in N/C (Newtons per Coulomb)
    """
    # Initialize total field
    total_E_field = np.zeros(3)
    
    # Sum the contributions from each charge
    for i, (charge, position) in enumerate(zip(charges, charge_positions)):
        E_field = electric_field(charge, position, field_point)
        total_E_field += E_field
        
    return total_E_field

if __name__ == "__main__":
    # Example usage with multiple charges
    
    # Define charges and their positions
    charges = [1e-9, 1e-9, 1e-9, 1e-9,1e-9,1e-9,1e-9,1e-9]  # Charges in Coulombs
    charge_positions = [
        [0, -1, 0],     # First charge at y=-1
        [1, 0, 0],     # Second charge at x=1 
        [0, 1, 0],     # Third charge at y=1
        [-1, 0, 0],     # Fourth Charge x=-1
        [0.707,0.707,0], 
        [-0.707,-0.707,0],
        [-0.707,0.707,0], 
        [0.707,-0.707,0]
    ]

    
    # Calculate field at a specific point
    field_point = [0, 0, 2]
    
    # Calculate electric field from each individual charge
    print("Electric field from individual charges:")
    for i, (charge, position) in enumerate(zip(charges, charge_positions)):
        E = electric_field(charge, position, field_point)
        print(f"Charge {i+1}: E = [{E[0]:.3}, {E[1]:.3}, {E[2]:.3}] N/C")
    
    # Calculate the total electric field
    total_E = electric_field_multiple_charges(charges, charge_positions, field_point)
    print("\nTotal electric field:")
    print(f"E = [{total_E[0]:.3}, {total_E[1]:.3}, {total_E[2]:.3}] N/C")
    print(f"Field magnitude: {np.linalg.norm(total_E):.4} N/C")

# Field from Uniformly-Charged Plate

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

# Reusing the existing functions from your code
def electric_field(charge, charge_position, field_point):
    # Convert inputs to numpy arrays
    charge_position = np.array(charge_position)
    field_point = np.array(field_point)
    
    # Calculate displacement vector from charge to field point
    r_vector = field_point - charge_position
    
    # Calculate distance
    r = np.linalg.norm(r_vector)
    
    # Check for singularity (field point at charge position)
    if r < 1e-10:
        return np.zeros(3)
    
    # Calculate unit vector
    r_hat = r_vector / r
    
    # Calculate electric field magnitude
    # E = kq/r², where k is Coulomb's constant
    k = 8.9875e9  # Coulomb's constant (N·m²/C²)
    E_magnitude = k * charge / (r**2)
    
    # Calculate electric field vector
    E_field = E_magnitude * r_hat
    
    return E_field

def electric_field_multiple_charges(charges, charge_positions, field_point):
    # Initialize total field
    total_E_field = np.zeros(3)
    
    # Sum the contributions from each charge
    for i, (charge, position) in enumerate(zip(charges, charge_positions)):
        E_field = electric_field(charge, position, field_point)
        total_E_field += E_field
        
    return total_E_field

def generate_charged_plate(width, height, nx, ny, charge_density, plate_orientation='xy', center_position=[0, 0, 0]):
    """
    Generate a rectangular plate of point charges.
    
    Parameters:
    -----------
    width : float
        Width of the plate in meters (first dimension)
    height : float
        Height of the plate in meters (second dimension)
    nx : int
        Number of charges along the width
    ny : int
        Number of charges along the height
    charge_density : float
        Surface charge density in C/m²
    plate_orientation : str
        Orientation of the plate ('xy', 'yz', or 'xz')
    center_position : list
        Center position of the plate [x, y, z]
        
    Returns:
    --------
    charges : numpy.ndarray
        Array of charge values for each point charge
    positions : numpy.ndarray
        Array of positions [x, y, z] for each point charge
    """
    # Calculate the area represented by each point charge
    dx = width / nx
    dy = height / ny
    area_per_charge = dx * dy
    
    # Calculate the charge value for each point
    charge_value = charge_density * area_per_charge
    
    charges = []
    positions = []
    
    center_position = np.array(center_position)
    
    # Create a grid of charges based on orientation
    if plate_orientation == 'xy':  # Plate in xy-plane
        for i in range(nx):
            for j in range(ny):
                # Calculate position (centered around specified position)
                x = (i + 0.5) * dx - width/2 + center_position[0]
                y = (j + 0.5) * dy - height/2 + center_position[1]
                z = center_position[2]
                
                charges.append(charge_value)
                positions.append([x, y, z])
                
    elif plate_orientation == 'yz':  # Plate in yz-plane
        for i in range(nx):
            for j in range(ny):
                # Calculate position (centered around specified position)
                y = (i + 0.5) * dx - width/2 + center_position[1]
                z = (j + 0.5) * dy - height/2 + center_position[2]
                x = center_position[0]
                
                charges.append(charge_value)
                positions.append([x, y, z])
                
    elif plate_orientation == 'xz':  # Plate in xz-plane
        for i in range(nx):
            for j in range(ny):
                # Calculate position (centered around specified position)
                x = (i + 0.5) * dx - width/2 + center_position[0]
                z = (j + 0.5) * dy - height/2 + center_position[2]
                y = center_position[1]
                
                charges.append(charge_value)
                positions.append([x, y, z])
    
    return np.array(charges), np.array(positions)

def calculate_field_parallel_plane(charges, charge_positions, plate_orientation='xy', 
                                  observation_distance=0.5, plane_size=(1.0, 1.0), 
                                  num_points=(30, 30)):
    """
    Calculate the electric field on a plane parallel to the charged plate.
    
    Parameters:
    -----------
    charges : numpy.ndarray
        Array of charge values
    charge_positions : numpy.ndarray
        Array of charge positions
    plate_orientation : str
        Orientation of the plate ('xy', 'yz', or 'xz')
    observation_distance : float
        Distance from the plate to the observation plane
    plane_size : tuple
        Size of the observation plane (width, height)
    num_points : tuple
        Resolution of the grid (points_width, points_height)
        
    Returns:
    --------
    grid1, grid2 : numpy.ndarray
        Mesh grid coordinates for the observation plane
    Ex, Ey, Ez : numpy.ndarray
        Components of the electric field
    E_mag : numpy.ndarray
        Magnitude of the electric field
    """
    # Determine the plate center position
    plate_center = np.mean(charge_positions, axis=0)
    
    # Create mesh grid for the observation plane
    if plate_orientation == 'xy':  # Plate in xy-plane, observation in xy-plane
        # Observation plane is parallel to xy-plane at z = plate_center[2] + observation_distance
        x = np.linspace(-plane_size[0]/2, plane_size[0]/2, num_points[0]) + plate_center[0]
        y = np.linspace(-plane_size[1]/2, plane_size[1]/2, num_points[1]) + plate_center[1]
        X, Y = np.meshgrid(x, y)
        Z = np.full_like(X, plate_center[2] + observation_distance)
        grid1, grid2 = X, Y  # First and second coordinates of the plane
        
    elif plate_orientation == 'yz':  # Plate in yz-plane, observation in yz-plane
        # Observation plane is parallel to yz-plane at x = plate_center[0] + observation_distance
        y = np.linspace(-plane_size[0]/2, plane_size[0]/2, num_points[0]) + plate_center[1]
        z = np.linspace(-plane_size[1]/2, plane_size[1]/2, num_points[1]) + plate_center[2]
        Y, Z = np.meshgrid(y, z)
        X = np.full_like(Y, plate_center[0] + observation_distance)
        grid1, grid2 = Y, Z  # First and second coordinates of the plane
        
    elif plate_orientation == 'xz':  # Plate in xz-plane, observation in xz-plane
        # Observation plane is parallel to xz-plane at y = plate_center[1] + observation_distance
        x = np.linspace(-plane_size[0]/2, plane_size[0]/2, num_points[0]) + plate_center[0]
        z = np.linspace(-plane_size[1]/2, plane_size[1]/2, num_points[1]) + plate_center[2]
        X, Z = np.meshgrid(x, z)
        Y = np.full_like(X, plate_center[1] + observation_distance)
        grid1, grid2 = X, Z  # First and second coordinates of the plane
    
    # Initialize field component arrays
    Ex = np.zeros_like(grid1)
    Ey = np.zeros_like(grid1)
    Ez = np.zeros_like(grid1)
    E_mag = np.zeros_like(grid1)
    
    # Calculate electric field at each point in the grid
    for i in range(num_points[1]):
        for j in range(num_points[0]):
            field_point = [X[i, j], Y[i, j], Z[i, j]]
                
            E = electric_field_multiple_charges(charges, charge_positions, field_point)
            
            Ex[i, j] = E[0]
            Ey[i, j] = E[1]
            Ez[i, j] = E[2]
            E_mag[i, j] = np.linalg.norm(E)
    
    return grid1, grid2, Ex, Ey, Ez, E_mag

def plot_field_parallel_plane(grid1, grid2, Ex, Ey, Ez, E_mag, plate_orientation='xy', observation_distance=0.5):
    """
    Plot the electric field on a plane parallel to the charged plate.
    
    Parameters:
    -----------
    grid1, grid2 : numpy.ndarray
        Mesh grid coordinates for the observation plane
    Ex, Ey, Ez : numpy.ndarray
        Components of the electric field
    E_mag : numpy.ndarray
        Magnitude of the electric field
    plate_orientation : str
        Orientation of the plate ('xy', 'yz', or 'xz')
    observation_distance : float
        Distance from the plate to the observation plane
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Get the components to plot based on orientation
    if plate_orientation == 'xy':
        # For xy-plane, show X and Y components, color by Z component
        comp1, comp2 = Ex, Ey  # Vector components to plot (x, y)
        perp_comp = Ez  # Perpendicular component (color)
        ax1_label, ax2_label = 'X', 'Y'
        perp_axis = 'Z'
        
    elif plate_orientation == 'yz':
        # For yz-plane, show Y and Z components, color by X component
        comp1, comp2 = Ey, Ez  # Vector components to plot (y, z)
        perp_comp = Ex  # Perpendicular component (color)
        ax1_label, ax2_label = 'Y', 'Z'
        perp_axis = 'X'
        
    elif plate_orientation == 'xz':
        # For xz-plane, show X and Z components, color by Y component
        comp1, comp2 = Ex, Ez  # Vector components to plot (x, z)
        perp_comp = Ey  # Perpendicular component (color)
        ax1_label, ax2_label = 'X', 'Z'
        perp_axis = 'Y'
    
    # Plot 1: Vector field of in-plane components
    ax1.set_title(f'Electric Field Vector Plot ({perp_axis}={observation_distance}m)')
    ax1.set_xlabel(f'{ax1_label} (m)')
    ax1.set_ylabel(f'{ax2_label} (m)')
    
    # Plot a subset of vectors for clarity
    skip = slice(None, None, 2)
    quiv = ax1.quiver(grid1[skip, skip], grid2[skip, skip], comp1[skip, skip], comp2[skip, skip], 
               angles='xy', scale_units='xy', scale=90000, width=0.003)
    
    # Add a key/legend for the quiver plot
    ax1.quiverkey(quiv, 0.9, 0.95, 50000, '50 kN/C', labelpos='E')
    
    ax1.grid(True)
    ax1.set_aspect('equal')
    
    # Plot 2: Perpendicular component as color map
    im = ax2.imshow(perp_comp, extent=[grid1.min(), grid1.max(), grid2.min(), grid2.max()], 
               origin='lower', cmap='viridis', aspect='auto')
    
    ax2.set_title(f'{perp_axis}-Component of Electric Field ({perp_axis}={observation_distance}m)')
    ax2.set_xlabel(f'{ax1_label} (m)')
    ax2.set_ylabel(f'{ax2_label} (m)')
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax2)
    cbar.set_label(f'{perp_axis}-Component (N/C)')
    
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return fig

if __name__ == "__main__":
    # Configuration
    plate_width = 2  # meters
    plate_height = 2  # meters
    nx = 30  # number of charges along width
    ny = 30  # number of charges along height
    charge_density = 1e-6  # C/m²
    
    # Set plate orientation - change this to 'xy', 'yz', or 'xz'
    plate_orientation = 'yz'  # Plate in the yz-plane (x=0)
    
    # Generate the charged plate
    charges, positions = generate_charged_plate(
        plate_width, plate_height, nx, ny, 
        charge_density,
        plate_orientation=plate_orientation,
        center_position=[0, 0, 0]  # Plate centered at origin
    )
    
    print(f"Generated {len(charges)} charges for the plate")
    print(f"Total charge: {np.sum(charges):.2e} C")
    
    # Set the observation distance from the plate
    observation_distance = 0.5  # meters
    
    # Calculate the field on the parallel plane
    grid1, grid2, Ex, Ey, Ez, E_mag = calculate_field_parallel_plane(
        charges, positions,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance,
        plane_size=(5.2, 5.2),  # Size of the observation plane (slightly larger than plate)
        num_points=(40, 40)  # Resolution (higher = more detail but slower)
    )
    
    # Plot the results
    fig = plot_field_parallel_plane(
        grid1, grid2, Ex, Ey, Ez, E_mag,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance
    )
    
    # You can change the observation distance to see how the field changes
    # For example:
    """
    # Change observation distance
    observation_distance = 0.5  # meters
    
    # Recalculate field
    grid1, grid2, Ex, Ey, Ez, E_mag = calculate_field_parallel_plane(
        charges, positions,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance,
        plane_size=(1.2, 1.2),
        num_points=(40, 40)
    )
    
    # Plot new results
    fig = plot_field_parallel_plane(
        grid1, grid2, Ex, Ey, Ez, E_mag,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance
    )
    """

# Field from Non-Uniformly Charged Plate (Dipole)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

# Reusing the existing functions from your code
def electric_field(charge, charge_position, field_point):
    """
    Calculate the electric field at a point due to a single point charge.
    
    Parameters:
    -----------
    charge : float
        Charge value in Coulombs
    charge_position : array-like
        Position of the charge [x, y, z] in meters
    field_point : array-like
        Position where the field is calculated [x, y, z] in meters
        
    Returns:
    --------
    E_field : numpy.ndarray
        Electric field vector [Ex, Ey, Ez] in N/C
    """
    # Convert inputs to numpy arrays
    charge_position = np.array(charge_position)
    field_point = np.array(field_point)
    
    # Calculate displacement vector from charge to field point
    r_vector = field_point - charge_position
    
    # Calculate distance
    r = np.linalg.norm(r_vector)
    
    # Check for singularity (field point at charge position)
    if r < 1e-10:
        return np.zeros(3)
    
    # Calculate unit vector
    r_hat = r_vector / r
    
    # Calculate electric field magnitude
    # E = kq/r², where k is Coulomb's constant
    k = 8.9875e9  # Coulomb's constant (N·m²/C²)
    E_magnitude = k * charge / (r**2)
    
    # Calculate electric field vector
    E_field = E_magnitude * r_hat
    
    return E_field

def electric_field_multiple_charges(charges, charge_positions, field_point):
    """
    Calculate the total electric field at a point due to multiple charges.
    
    Parameters:
    -----------
    charges : array-like
        Array of charge values in Coulombs
    charge_positions : array-like
        Array of charge positions, each [x, y, z] in meters
    field_point : array-like
        Position where the field is calculated [x, y, z] in meters
        
    Returns:
    --------
    total_E_field : numpy.ndarray
        Total electric field vector [Ex, Ey, Ez] in N/C
    """
    # Initialize total field
    total_E_field = np.zeros(3)
    
    # Sum the contributions from each charge
    for i, (charge, position) in enumerate(zip(charges, charge_positions)):
        E_field = electric_field(charge, position, field_point)
        total_E_field += E_field
        
    return total_E_field

def generate_charged_plate_non_uniform(width, height, nx, ny, charge_pattern='dipole', 
                                      plate_orientation='xy', center_position=[0, 0, 0], 
                                      pattern_params=None):
    """
    Generate a rectangular plate with non-uniform charge distribution.
    
    Parameters:
    -----------
    width : float
        Width of the plate in meters (first dimension)
    height : float
        Height of the plate in meters (second dimension)
    nx : int
        Number of charges along the width
    ny : int
        Number of charges along the height
    charge_pattern : str
        Type of charge distribution ('uniform', 'dipole', 'quadrupole', 'gaussian', 'custom')
    plate_orientation : str
        Orientation of the plate ('xy', 'yz', or 'xz')
    center_position : list
        Center position of the plate [x, y, z]
    pattern_params : dict
        Additional parameters for specific charge patterns:
        - uniform: {'max_density': float}
        - dipole: {'axis': 'x' or 'y', 'max_density': float}
        - quadrupole: {'max_density': float}
        - gaussian: {'sigma': float, 'amplitude': float}
        - custom: {'function': callable that takes (x, y) and returns charge density}
        
    Returns:
    --------
    charges : numpy.ndarray
        Array of charge values for each point charge
    positions : numpy.ndarray
        Array of positions [x, y, z] for each point charge
    """
    # Set default pattern parameters if not provided
    if pattern_params is None:
        if charge_pattern == 'uniform':
            pattern_params = {'max_density': 1e-6}
        elif charge_pattern == 'dipole':
            pattern_params = {'axis': 'y', 'max_density': 1e-6}
        elif charge_pattern == 'quadrupole':
            pattern_params = {'max_density': 1e-6}
        elif charge_pattern == 'gaussian':
            pattern_params = {'sigma': 0.2, 'amplitude': 1e-6}
        elif charge_pattern == 'custom':
            # Default to a simple linear gradient along y-axis
            pattern_params = {'function': lambda x, y: 1e-6 * y / (height/2)}
    
    # Calculate the area represented by each point charge
    dx = width / nx
    dy = height / ny
    area_per_charge = dx * dy
    
    charges = []
    positions = []
    
    center_position = np.array(center_position)
    
    # Create a grid of charges based on orientation
    for i in range(nx):
        for j in range(ny):
            # Calculate position relative to center
            rel_x = (i + 0.5) * dx - width/2
            rel_y = (j + 0.5) * dy - height/2
            
            # Determine charge value based on pattern
            if charge_pattern == 'uniform':
                max_density = pattern_params.get('max_density', 1e-6)
                # Uniform charge density (constant value)
                charge_density = max_density
            elif charge_pattern == 'dipole':
                axis = pattern_params.get('axis', 'y')
                max_density = pattern_params.get('max_density', 1e-6)
                
                if axis == 'x':
                    # Dipole along x-axis
                    charge_density = max_density * (rel_x / (width/2))
                else:  # 'y'
                    # Dipole along y-axis
                    charge_density = max_density * (rel_y / (height/2))
                    
            elif charge_pattern == 'quadrupole':
                max_density = pattern_params.get('max_density', 1e-6)
                # Quadrupole pattern (alternating charges)
                normalized_x = rel_x / (width/2)
                normalized_y = rel_y / (height/2)
                charge_density = max_density * normalized_x * normalized_y
                
            elif charge_pattern == 'gaussian':
                sigma = pattern_params.get('sigma', 0.2)
                amplitude = pattern_params.get('amplitude', 1e-6)
                # Gaussian positive charge at center
                r_squared = rel_x**2 + rel_y**2
                charge_density = amplitude * np.exp(-r_squared / (2 * sigma**2))
                
            elif charge_pattern == 'custom' and 'function' in pattern_params:
                # Use custom function to determine charge density
                charge_density = pattern_params['function'](rel_x, rel_y)
                
            else:
                raise ValueError(f"Unknown charge pattern: {charge_pattern}")
                
            # Calculate charge for this point
            charge_value = charge_density * area_per_charge
            
            # Set position based on plate orientation
            if plate_orientation == 'xy':  # Plate in xy-plane
                x = rel_x + center_position[0]
                y = rel_y + center_position[1]
                z = center_position[2]
                
            elif plate_orientation == 'yz':  # Plate in yz-plane
                y = rel_x + center_position[1]  # rel_x maps to y-axis
                z = rel_y + center_position[2]  # rel_y maps to z-axis
                x = center_position[0]
                
            elif plate_orientation == 'xz':  # Plate in xz-plane
                x = rel_x + center_position[0]  # rel_x maps to x-axis
                z = rel_y + center_position[2]  # rel_y maps to z-axis
                y = center_position[1]
                
            charges.append(charge_value)
            positions.append([x, y, z])
    
    return np.array(charges), np.array(positions)

def visualize_charge_distribution(charges, positions, plate_orientation='xy'):
    """
    Visualize the charge distribution on the plate.
    
    Parameters:
    -----------
    charges : numpy.ndarray
        Array of charge values
    positions : numpy.ndarray
        Array of charge positions
    plate_orientation : str
        Orientation of the plate ('xy', 'yz', or 'xz')
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111)
    
    # Extract the 2D coordinates based on plate orientation
    if plate_orientation == 'xy':
        x_coords = positions[:, 0]
        y_coords = positions[:, 1]
        x_label, y_label = 'X', 'Y'
    elif plate_orientation == 'yz':
        x_coords = positions[:, 1]  # Y coordinate
        y_coords = positions[:, 2]  # Z coordinate
        x_label, y_label = 'Y', 'Z'
    elif plate_orientation == 'xz':
        x_coords = positions[:, 0]  # X coordinate
        y_coords = positions[:, 2]  # Z coordinate
        x_label, y_label = 'X', 'Z'
    
    # Create a scatter plot with charge as color
    sc = ax.scatter(x_coords, y_coords, c=charges, cmap='RdBu_r', 
                   vmin=-np.max(np.abs(charges)), vmax=np.max(np.abs(charges)),
                   s=50, alpha=0.8)
    
    # Add colorbar
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label('Charge (C)')
    
    # Set labels
    ax.set_xlabel(f'{x_label} (m)')
    ax.set_ylabel(f'{y_label} (m)')
    ax.set_title('Charge Distribution on Plate')
    
    ax.grid(True)
    ax.set_aspect('equal')
    
    plt.tight_layout()
    
    return fig

def calculate_field_parallel_plane(charges, charge_positions, plate_orientation='xy', 
                                  observation_distance=0.5, plane_size=(1.0, 1.0), 
                                  num_points=(30, 30)):
    """
    Calculate the electric field on a plane parallel to the charged plate.
    
    Parameters:
    -----------
    charges : numpy.ndarray
        Array of charge values
    charge_positions : numpy.ndarray
        Array of charge positions
    plate_orientation : str
        Orientation of the plate ('xy', 'yz', or 'xz')
    observation_distance : float
        Distance from the plate to the observation plane
    plane_size : tuple
        Size of the observation plane (width, height)
    num_points : tuple
        Resolution of the grid (points_width, points_height)
        
    Returns:
    --------
    grid1, grid2 : numpy.ndarray
        Mesh grid coordinates for the observation plane
    Ex, Ey, Ez : numpy.ndarray
        Components of the electric field
    E_mag : numpy.ndarray
        Magnitude of the electric field
    """
    # Determine the plate center position
    plate_center = np.mean(charge_positions, axis=0)
    
    # Create mesh grid for the observation plane
    if plate_orientation == 'xy':  # Plate in xy-plane, observation in xy-plane
        # Observation plane is parallel to xy-plane at z = plate_center[2] + observation_distance
        x = np.linspace(-plane_size[0]/2, plane_size[0]/2, num_points[0]) + plate_center[0]
        y = np.linspace(-plane_size[1]/2, plane_size[1]/2, num_points[1]) + plate_center[1]
        X, Y = np.meshgrid(x, y)
        Z = np.full_like(X, plate_center[2] + observation_distance)
        grid1, grid2 = X, Y  # First and second coordinates of the plane
        
    elif plate_orientation == 'yz':  # Plate in yz-plane, observation in yz-plane
        # Observation plane is parallel to yz-plane at x = plate_center[0] + observation_distance
        y = np.linspace(-plane_size[0]/2, plane_size[0]/2, num_points[0]) + plate_center[1]
        z = np.linspace(-plane_size[1]/2, plane_size[1]/2, num_points[1]) + plate_center[2]
        Y, Z = np.meshgrid(y, z)
        X = np.full_like(Y, plate_center[0] + observation_distance)
        grid1, grid2 = Y, Z  # First and second coordinates of the plane
        
    elif plate_orientation == 'xz':  # Plate in xz-plane, observation in xz-plane
        # Observation plane is parallel to xz-plane at y = plate_center[1] + observation_distance
        x = np.linspace(-plane_size[0]/2, plane_size[0]/2, num_points[0]) + plate_center[0]
        z = np.linspace(-plane_size[1]/2, plane_size[1]/2, num_points[1]) + plate_center[2]
        X, Z = np.meshgrid(x, z)
        Y = np.full_like(X, plate_center[1] + observation_distance)
        grid1, grid2 = X, Z  # First and second coordinates of the plane
    
    # Initialize field component arrays
    Ex = np.zeros_like(grid1)
    Ey = np.zeros_like(grid1)
    Ez = np.zeros_like(grid1)
    E_mag = np.zeros_like(grid1)
    
    # Calculate electric field at each point in the grid
    for i in range(num_points[1]):
        for j in range(num_points[0]):
            field_point = [X[i, j], Y[i, j], Z[i, j]]
                
            E = electric_field_multiple_charges(charges, charge_positions, field_point)
            
            Ex[i, j] = E[0]
            Ey[i, j] = E[1]
            Ez[i, j] = E[2]
            E_mag[i, j] = np.linalg.norm(E)
    
    return grid1, grid2, Ex, Ey, Ez, E_mag

def plot_field_parallel_plane(grid1, grid2, Ex, Ey, Ez, E_mag, plate_orientation='xy', observation_distance=0.5):
    """
    Plot the electric field on a plane parallel to the charged plate.
    
    Parameters:
    -----------
    grid1, grid2 : numpy.ndarray
        Mesh grid coordinates for the observation plane
    Ex, Ey, Ez : numpy.ndarray
        Components of the electric field
    E_mag : numpy.ndarray
        Magnitude of the electric field
    plate_orientation : str
        Orientation of the plate ('xy', 'yz', or 'xz')
    observation_distance : float
        Distance from the plate to the observation plane
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Get the components to plot based on orientation
    if plate_orientation == 'xy':
        # For xy-plane, show X and Y components, color by Z component
        comp1, comp2 = Ex, Ey  # Vector components to plot (x, y)
        perp_comp = Ez  # Perpendicular component (color)
        ax1_label, ax2_label = 'X', 'Y'
        perp_axis = 'Z'
        
    elif plate_orientation == 'yz':
        # For yz-plane, show Y and Z components, color by X component
        comp1, comp2 = Ey, Ez  # Vector components to plot (y, z)
        perp_comp = Ex  # Perpendicular component (color)
        ax1_label, ax2_label = 'Y', 'Z'
        perp_axis = 'X'
        
    elif plate_orientation == 'xz':
        # For xz-plane, show X and Z components, color by Y component
        comp1, comp2 = Ex, Ez  # Vector components to plot (x, z)
        perp_comp = Ey  # Perpendicular component (color)
        ax1_label, ax2_label = 'X', 'Z'
        perp_axis = 'Y'
    
    # Calculate an appropriate scale for the quiver plot based on field strength
    max_comp = np.max(np.sqrt(comp1**2 + comp2**2))
    scale_value = max_comp * 2  # Adjust this multiplier as needed for visual clarity
    
    # Plot 1: Vector field of in-plane components
    ax1.set_title(f'Electric Field Vector Plot ({perp_axis}={observation_distance}m)')
    ax1.set_xlabel(f'{ax1_label} (m)')
    ax1.set_ylabel(f'{ax2_label} (m)')
    
    # Plot a subset of vectors for clarity
    skip = slice(None, None, 2)
    quiv = ax1.quiver(grid1[skip, skip], grid2[skip, skip], comp1[skip, skip], comp2[skip, skip], 
               angles='xy', scale_units='xy', scale=scale_value, width=0.003)
    
    # Add a key/legend for the quiver plot
    key_value = round(max_comp / 2, -int(np.floor(np.log10(max_comp / 2))))  # Round to nice number
    ax1.quiverkey(quiv, 0.9, 0.95, key_value, f'{key_value} N/C', labelpos='E')
    
    ax1.grid(True)
    ax1.set_aspect('equal')
    
    # Plot 2: Perpendicular component as color map
    # Use a symmetric colormap centered at zero for better visualization of positive/negative fields
    max_val = np.max(np.abs(perp_comp))
    norm = Normalize(vmin=-max_val, vmax=max_val)
    
    im = ax2.imshow(perp_comp, extent=[grid1.min(), grid1.max(), grid2.min(), grid2.max()], 
               origin='lower', cmap='RdBu_r', norm=norm, aspect='auto')
    
    ax2.set_title(f'{perp_axis}-Component of Electric Field ({perp_axis}={observation_distance}m)')
    ax2.set_xlabel(f'{ax1_label} (m)')
    ax2.set_ylabel(f'{ax2_label} (m)')
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax2)
    cbar.set_label(f'{perp_axis}-Component (N/C)')
    
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return fig

if __name__ == "__main__":
    # Configuration
    plate_width = 2.0  # meters
    plate_height = 2.0  # meters
    nx = 40  # number of charges along width
    ny = 40  # number of charges along height
    
    # Set plate orientation - change this to 'xy', 'yz', or 'xz'
    plate_orientation = 'yz'  # Plate in the yz-plane (x=0)
    
    # UNIFORM PATTERN - only keep this section uncommented
    charge_pattern = 'uniform'
    pattern_params = {'max_density': 1e-6}  # Uniform charge density
    
    charges, positions = generate_charged_plate_non_uniform(
        plate_width, plate_height, nx, ny,
        charge_pattern=charge_pattern,
        pattern_params=pattern_params,
        plate_orientation=plate_orientation,
        center_position=[0, 0, 0]  # Plate centered at origin
    )
    
    print(f"Generated {len(charges)} charges for the plate")
    print(f"Net charge: {np.sum(charges):.2e} C")
    print(f"Charge range: {np.min(charges):.2e} to {np.max(charges):.2e} C")
    
    # Visualize the charge distribution
    visualize_charge_distribution(charges, positions, plate_orientation)
    
    # Set the observation distance from the plate
    observation_distance = 0.5  # meters
    
    # Calculate the field on the parallel plane
    grid1, grid2, Ex, Ey, Ez, E_mag = calculate_field_parallel_plane(
        charges, positions,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance,
        plane_size=(2.5, 2.5),  # Size of the observation plane (slightly larger than plate)
        num_points=(40, 40)  # Resolution (higher = more detail but slower)
    )
    
    # Plot the results
    fig = plot_field_parallel_plane(
        grid1, grid2, Ex, Ey, Ez, E_mag,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance
    )
    
    # ===== COMMENT OUT ALL SECTIONS BELOW THIS LINE IF YOU ONLY WANT UNIFORM DISTRIBUTION =====
    
    '''
    # DIPOLE PATTERN - START OF COMMENT BLOCK
    charge_pattern = 'dipole'
    pattern_params = {
        'axis': 'y',  # Dipole along y-axis
        'max_density': 1e-6  # Maximum charge density (C/m²)
    }
    
    charges, positions = generate_charged_plate_non_uniform(
        plate_width, plate_height, nx, ny,
        charge_pattern=charge_pattern,
        pattern_params=pattern_params,
        plate_orientation=plate_orientation,
        center_position=[0, 0, 0]
    )
    
    visualize_charge_distribution(charges, positions, plate_orientation)
    
    grid1, grid2, Ex, Ey, Ez, E_mag = calculate_field_parallel_plane(
        charges, positions,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance,
        plane_size=(2.5, 2.5),
        num_points=(40, 40)
    )
    
    plot_field_parallel_plane(
        grid1, grid2, Ex, Ey, Ez, E_mag,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance
    )
    # DIPOLE PATTERN - END OF COMMENT BLOCK
    
    # QUADRUPOLE PATTERN - START OF COMMENT BLOCK
    charge_pattern = 'quadrupole'
    pattern_params = {'max_density': 1e-6}
    
    charges, positions = generate_charged_plate_non_uniform(
        plate_width, plate_height, nx, ny,
        charge_pattern=charge_pattern,
        pattern_params=pattern_params,
        plate_orientation=plate_orientation,
        center_position=[0, 0, 0]
    )
    
    visualize_charge_distribution(charges, positions, plate_orientation)
    
    grid1, grid2, Ex, Ey, Ez, E_mag = calculate_field_parallel_plane(
        charges, positions,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance,
        plane_size=(2.5, 2.5),
        num_points=(40, 40)
    )
    
    plot_field_parallel_plane(
        grid1, grid2, Ex, Ey, Ez, E_mag,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance
    )
    # QUADRUPOLE PATTERN - END OF COMMENT BLOCK

    # GAUSSIAN PATTERN - START OF COMMENT BLOCK
    charge_pattern = 'gaussian'
    pattern_params = {'sigma': 0.4, 'amplitude': 1e-6}
    
    charges, positions = generate_charged_plate_non_uniform(
        plate_width, plate_height, nx, ny,
        charge_pattern=charge_pattern,
        pattern_params=pattern_params,
        plate_orientation=plate_orientation,
        center_position=[0, 0, 0]
    )
    
    visualize_charge_distribution(charges, positions, plate_orientation)
    
    grid1, grid2, Ex, Ey, Ez, E_mag = calculate_field_parallel_plane(
        charges, positions,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance,
        plane_size=(2.5, 2.5),
        num_points=(40, 40)
    )
    
    plot_field_parallel_plane(
        grid1, grid2, Ex, Ey, Ez, E_mag,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance
    )
    # GAUSSIAN PATTERN - END OF COMMENT BLOCK
    
    # CUSTOM PATTERN - START OF COMMENT BLOCK
    def custom_charge_function(x, y):
        r1 = np.sqrt((x - 0.5)**2 + y**2)
        r2 = np.sqrt((x + 0.5)**2 + y**2)
        q1 = 1e-6 * np.exp(-r1**2 / 0.1)  # Positive charge at (0.5, 0)
        q2 = -1e-6 * np.exp(-r2**2 / 0.1)  # Negative charge at (-0.5, 0)
        return q1 + q2
    
    charge_pattern = 'custom'
    pattern_params = {'function': custom_charge_function}
    
    charges, positions = generate_charged_plate_non_uniform(
        plate_width, plate_height, nx, ny,
        charge_pattern=charge_pattern,
        pattern_params=pattern_params,
        plate_orientation=plate_orientation,
        center_position=[0, 0, 0]
    )
    
    visualize_charge_distribution(charges, positions, plate_orientation)
    
    grid1, grid2, Ex, Ey, Ez, E_mag = calculate_field_parallel_plane(
        charges, positions,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance,
        plane_size=(2.5, 2.5),
        num_points=(40, 40)
    )
    
    plot_field_parallel_plane(
        grid1, grid2, Ex, Ey, Ez, E_mag,
        plate_orientation=plate_orientation,
        observation_distance=observation_distance
    )
    # CUSTOM PATTERN - END OF COMMENT BLOCK
    '''

# Capacitor Field Cross Section


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

# Constants
k = 8.9875e9  # Coulomb's constant (N·m²/C²)

def electric_field(charge, charge_position, field_point):
    """
    Calculate the electric field at a point due to a single point charge.
    """
    # Convert inputs to numpy arrays
    charge_position = np.array(charge_position)
    field_point = np.array(field_point)
    
    # Calculate displacement vector from charge to field point
    r_vector = field_point - charge_position
    
    # Calculate distance
    r = np.linalg.norm(r_vector)
    
    # Check for singularity (field point at charge position)
    if r < 1e-10:
        return np.zeros(3)
    
    # Calculate unit vector
    r_hat = r_vector / r
    
    # Calculate electric field magnitude
    E_magnitude = k * charge / (r**2)
    
    # Calculate electric field vector
    E_field = E_magnitude * r_hat
    
    return E_field

def electric_field_multiple_charges(charges, charge_positions, field_point):
    """
    Calculate the total electric field at a point due to multiple charges.
    """
    # Initialize total field
    total_E_field = np.zeros(3)
    
    # Sum the contributions from each charge
    for i, (charge, position) in enumerate(zip(charges, charge_positions)):
        E_field = electric_field(charge, position, field_point)
        total_E_field += E_field
        
    return total_E_field

def generate_charged_plate(width, height, nx, ny, 
                          plate_orientation='xy', center_position=[0, 0, 0],
                          charge_density=1e-6):
    """
    Generate a uniformly charged rectangular plate.
    """
    # Calculate the area represented by each point charge
    dx = width / nx
    dy = height / ny
    area_per_charge = dx * dy
    
    # Calculate charge per point based on uniform density
    charge_value = charge_density * area_per_charge
    
    charges = []
    positions = []
    
    center_position = np.array(center_position)
    
    # Create a grid of charges based on orientation
    for i in range(nx):
        for j in range(ny):
            # Calculate position relative to center
            rel_x = (i + 0.5) * dx - width/2
            rel_y = (j + 0.5) * dy - height/2
            
            # Set position based on plate orientation
            if plate_orientation == 'xy':  # Plate in xy-plane
                x = rel_x + center_position[0]
                y = rel_y + center_position[1]
                z = center_position[2]
                
            elif plate_orientation == 'yz':  # Plate in yz-plane
                y = rel_x + center_position[1]  # rel_x maps to y-axis
                z = rel_y + center_position[2]  # rel_y maps to z-axis
                x = center_position[0]
                
            elif plate_orientation == 'xz':  # Plate in xz-plane
                x = rel_x + center_position[0]  # rel_x maps to x-axis
                z = rel_y + center_position[2]  # rel_y maps to z-axis
                y = center_position[1]
                
            charges.append(charge_value)
            positions.append([x, y, z])
    
    return np.array(charges), np.array(positions)

def generate_parallel_plates(width, height, nx, ny, plate_separation, 
                            plate_orientation='xy', center_position=[0, 0, 0],
                            charge_density=1e-6):
    """
    Generate two parallel uniformly charged plates with opposite charges.
    """
    center_position = np.array(center_position)
    
    # Calculate the centers of the two plates
    if plate_orientation == 'xy':
        # Plates in xy-plane, separation along z-axis
        plate1_center = center_position + np.array([0, 0, plate_separation/2])
        plate2_center = center_position + np.array([0, 0, -plate_separation/2])
        normal_axis = 'z'
    
    elif plate_orientation == 'yz':
        # Plates in yz-plane, separation along x-axis
        plate1_center = center_position + np.array([plate_separation/2, 0, 0])
        plate2_center = center_position + np.array([-plate_separation/2, 0, 0])
        normal_axis = 'x'
    
    elif plate_orientation == 'xz':
        # Plates in xz-plane, separation along y-axis
        plate1_center = center_position + np.array([0, plate_separation/2, 0])
        plate2_center = center_position + np.array([0, -plate_separation/2, 0])
        normal_axis = 'y'
    
    # Generate the first plate (positive charge)
    charges1, positions1 = generate_charged_plate(
        width, height, nx, ny,
        plate_orientation=plate_orientation,
        center_position=plate1_center,
        charge_density=charge_density
    )
    
    # Generate the second plate (negative charge)
    charges2, positions2 = generate_charged_plate(
        width, height, nx, ny,
        plate_orientation=plate_orientation,
        center_position=plate2_center,
        charge_density=-charge_density  # Negative charge
    )
    
    # Combine charges and positions from both plates
    all_charges = np.concatenate((charges1, charges2))
    all_positions = np.concatenate((positions1, positions2))
    
    return all_charges, all_positions, plate1_center, plate2_center, normal_axis

def calculate_field_cross_section(charges, charge_positions, 
                                slice_plane='xz', slice_position=0.0,
                                slice_size=(3.0, 3.0), num_points=(60, 60)):
    """
    Calculate the electric field on a cross-section slice.
    """
    # Create mesh grid for the slice
    if slice_plane == 'xy':  # Slice in xy-plane at z=slice_position
        x = np.linspace(-slice_size[0]/2, slice_size[0]/2, num_points[0])
        y = np.linspace(-slice_size[1]/2, slice_size[1]/2, num_points[1])
        X, Y = np.meshgrid(x, y)
        Z = np.full_like(X, slice_position)
        grid1, grid2 = X, Y
        perpendicular_axis = 'z'
        
    elif slice_plane == 'yz':  # Slice in yz-plane at x=slice_position
        y = np.linspace(-slice_size[0]/2, slice_size[0]/2, num_points[0])
        z = np.linspace(-slice_size[1]/2, slice_size[1]/2, num_points[1])
        Y, Z = np.meshgrid(y, z)
        X = np.full_like(Y, slice_position)
        grid1, grid2 = Y, Z
        perpendicular_axis = 'x'
        
    elif slice_plane == 'xz':  # Slice in xz-plane at y=slice_position
        x = np.linspace(-slice_size[0]/2, slice_size[0]/2, num_points[0])
        z = np.linspace(-slice_size[1]/2, slice_size[1]/2, num_points[1])
        X, Z = np.meshgrid(x, z)
        Y = np.full_like(X, slice_position)
        grid1, grid2 = X, Z
        perpendicular_axis = 'y'
    
    # Initialize field component arrays
    Ex = np.zeros_like(grid1)
    Ey = np.zeros_like(grid1)
    Ez = np.zeros_like(grid1)
    E_mag = np.zeros_like(grid1)
    
    # Calculate electric field at each point in the grid
    for i in range(num_points[1]):
        for j in range(num_points[0]):
            if slice_plane == 'xy':
                field_point = [X[i, j], Y[i, j], Z[i, j]]
            elif slice_plane == 'yz':
                field_point = [X[i, j], Y[i, j], Z[i, j]]
            elif slice_plane == 'xz':
                field_point = [X[i, j], Y[i, j], Z[i, j]]
                
            E = electric_field_multiple_charges(charges, charge_positions, field_point)
            
            Ex[i, j] = E[0]
            Ey[i, j] = E[1]
            Ez[i, j] = E[2]
            E_mag[i, j] = np.linalg.norm(E)
    
    return grid1, grid2, Ex, Ey, Ez, E_mag, perpendicular_axis

def visualize_cross_section_vectors(charges, positions, plate1_center, plate2_center, normal_axis,
                                   plate_width, plate_height, plate_orientation, slice_plane):
    """
    Visualize a single cross-section through the middle of the plates,
    representing the plates as infinitesimally thin lines and the electric field as unit vectors.
    """
    # Determine the slice position (middle of the capacitor)
    slice_position = 0.0  # Center position
    
    # Calculate fields on the cross-section
    grid1, grid2, Ex, Ey, Ez, E_mag, perp_axis = calculate_field_cross_section(
        charges, positions,
        slice_plane=slice_plane,
        slice_position=slice_position,
        slice_size=(plate_width * 2.5, plate_height * 2.5),  # Make the view area larger than plates
        num_points=(20, 20)  # Lower resolution for clearer vectors
    )
    
    # Create a figure with a single axes
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Get the components to plot based on orientation
    if slice_plane == 'xy':
        comp1, comp2 = Ex, Ey  # Vector components to plot
        ax1_label, ax2_label = 'X', 'Y'
        axis1_idx, axis2_idx = 0, 1  # Indices for extracting plate coordinates
        
    elif slice_plane == 'yz':
        comp1, comp2 = Ey, Ez  # Vector components to plot
        ax1_label, ax2_label = 'Y', 'Z'
        axis1_idx, axis2_idx = 1, 2
        
    elif slice_plane == 'xz':
        comp1, comp2 = Ex, Ez  # Vector components to plot
        ax1_label, ax2_label = 'X', 'Z'
        axis1_idx, axis2_idx = 0, 2
    
    
    # Calculate unit vectors for the quiver plot
    unit_comp1 = np.zeros_like(comp1)
    unit_comp2 = np.zeros_like(comp2)
    
    # Avoid division by zero
    mag = np.sqrt(comp1**2 + comp2**2)
    mask = mag > 1e-10
    
    unit_comp1[mask] = comp1[mask] / mag[mask]
    unit_comp2[mask] = comp2[mask] / mag[mask]
    
    # Create a quiver plot with unit vectors
    skip = 1  # Adjust to change vector density
    quiv = ax.quiver(grid1[::skip, ::skip], grid2[::skip, ::skip], 
                   unit_comp1[::skip, ::skip], unit_comp2[::skip, ::skip],
                   E_mag[::skip, ::skip],  # Color by magnitude
                   cmap='viridis',
                   scale=25,  # Adjust for vector size
                   width=0.004,  # Width of arrow
                   headwidth=3,  # Head width
                   pivot='mid')  # Arrow pivots at midpoint
    
    # Add colorbar for the field magnitude
    cbar = plt.colorbar(quiv, ax=ax)
    cbar.set_label('Electric Field Magnitude (N/C)')
    
    # Draw the plates as lines
    if plate_orientation == 'xy' and slice_plane == 'xz':
        # For xy plates viewed in xz slice, draw the edges of the plates as lines
        plate1_z = plate1_center[2]
        plate2_z = plate2_center[2]
        x_min = -plate_width/2
        x_max = plate_width/2
        
        # Top plate edge (positive)
        ax.plot([x_min, x_max], [plate1_z, plate1_z], 'r-', linewidth=3, label='Positive Plate (+)')
        # Add + symbols along the plate
        num_symbols = 10
        for i in range(num_symbols):
            x_pos = x_min + (x_max - x_min) * (i + 0.5) / num_symbols
            ax.text(x_pos, plate1_z, '+', color='red', fontsize=14, 
                   ha='center', va='center', fontweight='bold')
        
        # Bottom plate edge (negative)
        ax.plot([x_min, x_max], [plate2_z, plate2_z], 'b-', linewidth=3, label='Negative Plate (-)')
        # Add - symbols along the plate
        for i in range(num_symbols):
            x_pos = x_min + (x_max - x_min) * (i + 0.5) / num_symbols
            ax.text(x_pos, plate2_z, '−', color='blue', fontsize=14, 
                   ha='center', va='center', fontweight='bold')
    
    elif plate_orientation == 'xy' and slice_plane == 'yz':
        # For xy plates viewed in yz slice, draw the edges of the plates as lines
        plate1_z = plate1_center[2]
        plate2_z = plate2_center[2]
        y_min = -plate_height/2
        y_max = plate_height/2
        
        # Top plate edge (positive)
        ax.plot([y_min, y_max], [plate1_z, plate1_z], 'r-', linewidth=3, label='Positive Plate (+)')
        # Add + symbols along the plate
        num_symbols = 5
        for i in range(num_symbols):
            y_pos = y_min + (y_max - y_min) * (i + 0.5) / num_symbols
            ax.text(y_pos, plate1_z, '+', color='red', fontsize=14, 
                   ha='center', va='center', fontweight='bold')
        
        # Bottom plate edge (negative)
        ax.plot([y_min, y_max], [plate2_z, plate2_z], 'b-', linewidth=3, label='Negative Plate (-)')
        # Add - symbols along the plate
        for i in range(num_symbols):
            y_pos = y_min + (y_max - y_min) * (i + 0.5) / num_symbols
            ax.text(y_pos, plate2_z, '−', color='blue', fontsize=14, 
                   ha='center', va='center', fontweight='bold')
    
    # Add gridlines
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Set labels and title
    ax.set_xlabel(f'{ax1_label} (m)')
    ax.set_ylabel(f'{ax2_label} (m)')
    ax.set_title(f'Electric Field Cross-Section Through Capacitor: {slice_plane.upper()}-Plane (Unit Vectors)')
    
    # Set aspect ratio
    ax.set_aspect('equal')
    
    # Add legend
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    
    return fig

def main():
    """
    Main function to run the capacitor simulation and visualization.
    """
    # Configuration
    plate_width = 1.0  # meters
    plate_height = 1.0  # meters
    plate_separation = 0.5  # meters
    
    # Number of charges to represent each plate
    nx = 20  # points along width
    ny = 20  # points along height
    
    # Charge density (C/m²)
    charge_density = 1e-6
    
    # Orientation of the capacitor plates
    # Options: 'xy', 'yz', or 'xz'
    plate_orientation = 'xy'
    
    # Generate the capacitor plates
    charges, positions, plate1_center, plate2_center, normal_axis = generate_parallel_plates(
        plate_width, plate_height, nx, ny, 
        plate_separation=plate_separation,
        plate_orientation=plate_orientation,
        center_position=[0, 0, 0],
        charge_density=charge_density
    )
    
    print(f"Generated capacitor with {len(charges)} charges")
    print(f"Plate orientation: {plate_orientation}-plane")
    print(f"Plate separation: {plate_separation} m")
    print(f"Plate 1 center (positive): {plate1_center}")
    print(f"Plate 2 center (negative): {plate2_center}")
    
    # For xy-oriented plates, choose xz plane to see the cross-section
    if plate_orientation == 'xy':
        slice_plane = 'xz'  # slice along x-z plane to see the field between plates
    elif plate_orientation == 'yz':
        slice_plane = 'xz'  # slice along x-z plane
    elif plate_orientation == 'xz':
        slice_plane = 'xy'  # slice along x-y plane
        
    # Visualize the cross-section with unit vectors
    fig = visualize_cross_section_vectors(
        charges, positions, 
        plate1_center, plate2_center, 
        normal_axis,
        plate_width, plate_height, 
        plate_orientation, 
        slice_plane
    )
    
    plt.show()

if __name__ == "__main__":
    main()

# Capacitor w/ Dielectric 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

# Constants
k = 8.9875e9  # Coulomb's constant (N·m²/C²)

def electric_field(charge, charge_position, field_point):
    """
    Calculate the electric field at a point due to a single point charge.
    """
    # Convert inputs to numpy arrays
    charge_position = np.array(charge_position)
    field_point = np.array(field_point)
    
    # Calculate displacement vector from charge to field point
    r_vector = field_point - charge_position
    
    # Calculate distance
    r = np.linalg.norm(r_vector)
    
    # Check for singularity (field point at charge position)
    if r < 1e-10:
        return np.zeros(3)
    
    # Calculate unit vector
    r_hat = r_vector / r
    
    # Calculate electric field magnitude
    E_magnitude = k * charge / (r**2)
    
    # Calculate electric field vector
    E_field = E_magnitude * r_hat
    
    return E_field

def electric_field_multiple_charges(charges, charge_positions, field_point):
    """
    Calculate the total electric field at a point due to multiple charges.
    """
    # Initialize total field
    total_E_field = np.zeros(3)
    
    # Sum the contributions from each charge
    for i, (charge, position) in enumerate(zip(charges, charge_positions)):
        E_field = electric_field(charge, position, field_point)
        total_E_field += E_field
        
    return total_E_field

def generate_charged_plate(width, height, nx, ny, 
                          plate_orientation='xy', center_position=[0, 0, 0],
                          charge_density=1e-6):
    """
    Generate a uniformly charged rectangular plate.
    """
    # Calculate the area represented by each point charge
    dx = width / nx
    dy = height / ny
    area_per_charge = dx * dy
    
    # Calculate charge per point based on uniform density
    charge_value = charge_density * area_per_charge
    
    charges = []
    positions = []
    
    center_position = np.array(center_position)
    
    # Create a grid of charges based on orientation
    for i in range(nx):
        for j in range(ny):
            # Calculate position relative to center
            rel_x = (i + 0.5) * dx - width/2
            rel_y = (j + 0.5) * dy - height/2
            
            # Set position based on plate orientation
            if plate_orientation == 'xy':  # Plate in xy-plane
                x = rel_x + center_position[0]
                y = rel_y + center_position[1]
                z = center_position[2]
                
            elif plate_orientation == 'yz':  # Plate in yz-plane
                y = rel_x + center_position[1]  # rel_x maps to y-axis
                z = rel_y + center_position[2]  # rel_y maps to z-axis
                x = center_position[0]
                
            elif plate_orientation == 'xz':  # Plate in xz-plane
                x = rel_x + center_position[0]  # rel_x maps to x-axis
                z = rel_y + center_position[2]  # rel_y maps to z-axis
                y = center_position[1]
                
            charges.append(charge_value)
            positions.append([x, y, z])
    
    return np.array(charges), np.array(positions)

def generate_parallel_plates(width, height, nx, ny, plate_separation, 
                            plate_orientation='xy', center_position=[0, 0, 0],
                            charge_density=1e-6):
    """
    Generate two parallel uniformly charged plates with opposite charges.
    """
    center_position = np.array(center_position)
    
    # Calculate the centers of the two plates
    if plate_orientation == 'xy':
        # Plates in xy-plane, separation along z-axis
        plate1_center = center_position + np.array([0, 0, plate_separation/2])
        plate2_center = center_position + np.array([0, 0, -plate_separation/2])
        normal_axis = 'z'
    
    elif plate_orientation == 'yz':
        # Plates in yz-plane, separation along x-axis
        plate1_center = center_position + np.array([plate_separation/2, 0, 0])
        plate2_center = center_position + np.array([-plate_separation/2, 0, 0])
        normal_axis = 'x'
    
    elif plate_orientation == 'xz':
        # Plates in xz-plane, separation along y-axis
        plate1_center = center_position + np.array([0, plate_separation/2, 0])
        plate2_center = center_position + np.array([0, -plate_separation/2, 0])
        normal_axis = 'y'
    
    # Generate the first plate (positive charge)
    charges1, positions1 = generate_charged_plate(
        width, height, nx, ny,
        plate_orientation=plate_orientation,
        center_position=plate1_center,
        charge_density=charge_density
    )
    
    # Generate the second plate (negative charge)
    charges2, positions2 = generate_charged_plate(
        width, height, nx, ny,
        plate_orientation=plate_orientation,
        center_position=plate2_center,
        charge_density=-charge_density  # Negative charge
    )
    
    # Combine charges and positions from both plates
    all_charges = np.concatenate((charges1, charges2))
    all_positions = np.concatenate((positions1, positions2))
    
    return all_charges, all_positions, plate1_center, plate2_center, normal_axis

def generate_dielectric_plate(width, height, nx, ny, 
                          plate_orientation='xy', center_position=[0, 0, 0],
                          surface_charge_density=1e-7, thickness=0.05):
    """
    Generate a dielectric plate with surface charges due to polarization.
    Similar to generate_charged_plate but creates two surfaces with opposite charges.
    """
    # Calculate the area represented by each point charge
    dx = width / nx
    dy = height / ny
    area_per_charge = dx * dy
    
    # Calculate charge per point based on surface charge density
    charge_value = surface_charge_density * area_per_charge
    
    # We'll create two sets of charges - one for each surface of the dielectric
    charges_positive = []
    charges_negative = []
    positions_positive = []
    positions_negative = []
    
    # Separation between charge layers is the thickness of the dielectric
    surface_separation = thickness
    
    center_position = np.array(center_position)
    
    # Create a grid of charges based on orientation
    for i in range(nx):
        for j in range(ny):
            # Calculate position relative to center
            rel_x = (i + 0.5) * dx - width/2
            rel_y = (j + 0.5) * dy - height/2
            
            # Set positions based on plate orientation
            if plate_orientation == 'xy':  # Plate in xy-plane
                x = rel_x + center_position[0]
                y = rel_y + center_position[1]
                z_pos = center_position[2] + surface_separation/2
                z_neg = center_position[2] - surface_separation/2
                
                pos_positive = [x, y, z_pos]
                pos_negative = [x, y, z_neg]
                
            elif plate_orientation == 'yz':  # Plate in yz-plane
                y = rel_x + center_position[1]
                z = rel_y + center_position[2]
                x_pos = center_position[0] + surface_separation/2
                x_neg = center_position[0] - surface_separation/2
                
                pos_positive = [x_pos, y, z]
                pos_negative = [x_neg, y, z]
                
            elif plate_orientation == 'xz':  # Plate in xz-plane
                x = rel_x + center_position[0]
                z = rel_y + center_position[2]
                y_pos = center_position[1] + surface_separation/2
                y_neg = center_position[1] - surface_separation/2
                
                pos_positive = [x, y_pos, z]
                pos_negative = [x, y_neg, z]
            
            # Add charges and positions
            charges_positive.append(charge_value)
            charges_negative.append(-charge_value)
            positions_positive.append(pos_positive)
            positions_negative.append(pos_negative)
    
    # Combine all charges and positions
    all_charges = np.concatenate((charges_positive, charges_negative))
    all_positions = np.array(positions_positive + positions_negative)
    
    return all_charges, all_positions, surface_separation, positions_positive, positions_negative

def calculate_field_cross_section(charges, charge_positions, 
                                slice_plane='xz', slice_position=0.0,
                                slice_size=(3.0, 3.0), num_points=(60, 60)):
    """
    Calculate the electric field on a cross-section slice.
    """
    # Create mesh grid for the slice
    if slice_plane == 'xy':  # Slice in xy-plane at z=slice_position
        x = np.linspace(-slice_size[0]/2, slice_size[0]/2, num_points[0])
        y = np.linspace(-slice_size[1]/2, slice_size[1]/2, num_points[1])
        X, Y = np.meshgrid(x, y)
        Z = np.full_like(X, slice_position)
        grid1, grid2 = X, Y
        perpendicular_axis = 'z'
        
    elif slice_plane == 'yz':  # Slice in yz-plane at x=slice_position
        y = np.linspace(-slice_size[0]/2, slice_size[0]/2, num_points[0])
        z = np.linspace(-slice_size[1]/2, slice_size[1]/2, num_points[1])
        Y, Z = np.meshgrid(y, z)
        X = np.full_like(Y, slice_position)
        grid1, grid2 = Y, Z
        perpendicular_axis = 'x'
        
    elif slice_plane == 'xz':  # Slice in xz-plane at y=slice_position
        x = np.linspace(-slice_size[0]/2, slice_size[0]/2, num_points[0])
        z = np.linspace(-slice_size[1]/2, slice_size[1]/2, num_points[1])
        X, Z = np.meshgrid(x, z)
        Y = np.full_like(X, slice_position)
        grid1, grid2 = X, Z
        perpendicular_axis = 'y'
    
    # Initialize field component arrays
    Ex = np.zeros_like(grid1)
    Ey = np.zeros_like(grid1)
    Ez = np.zeros_like(grid1)
    E_mag = np.zeros_like(grid1)
    
    # Calculate electric field at each point in the grid
    for i in range(num_points[1]):
        for j in range(num_points[0]):
            if slice_plane == 'xy':
                field_point = [X[i, j], Y[i, j], Z[i, j]]
            elif slice_plane == 'yz':
                field_point = [X[i, j], Y[i, j], Z[i, j]]
            elif slice_plane == 'xz':
                field_point = [X[i, j], Y[i, j], Z[i, j]]
                
            E = electric_field_multiple_charges(charges, charge_positions, field_point)
            
            Ex[i, j] = E[0]
            Ey[i, j] = E[1]
            Ez[i, j] = E[2]
            E_mag[i, j] = np.linalg.norm(E)
    
    return grid1, grid2, Ex, Ey, Ez, E_mag, perpendicular_axis

def visualize_cross_section_vectors(charges, positions, plate1_center, plate2_center, 
                                   dielectric_center, dielectric_thickness, normal_axis,
                                   plate_width, plate_height, plate_orientation, slice_plane,
                                   pos_surface_positions, neg_surface_positions):
    """
    Visualize a single cross-section through the middle of the plates,
    representing the plates as infinitesimally thin lines and the electric field as unit vectors.
    """
    # Determine the slice position (middle of the capacitor)
    slice_position = 0.0  # Center position
    
    # Calculate fields on the cross-section - focus more narrowly on the capacitor region
    view_size_x = plate_width * 1.2  # Only slightly wider than the plates
    view_size_z = plate_separation * 1.5  # Only slightly taller than the separation
    
    grid1, grid2, Ex, Ey, Ez, E_mag, perp_axis = calculate_field_cross_section(
        charges, positions,
        slice_plane=slice_plane,
        slice_position=slice_position,
        slice_size=(view_size_x, view_size_z),  # Focused view
        num_points=(40, 40)  # Higher resolution for better field visualization
    )
    
    # Create a figure with a single axes
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Get the components to plot based on orientation
    if slice_plane == 'xy':
        comp1, comp2 = Ex, Ey  # Vector components to plot
        ax1_label, ax2_label = 'X', 'Y'
        axis1_idx, axis2_idx = 0, 1  # Indices for extracting plate coordinates
        
    elif slice_plane == 'yz':
        comp1, comp2 = Ey, Ez  # Vector components to plot
        ax1_label, ax2_label = 'Y', 'Z'
        axis1_idx, axis2_idx = 1, 2
        
    elif slice_plane == 'xz':
        comp1, comp2 = Ex, Ez  # Vector components to plot
        ax1_label, ax2_label = 'X', 'Z'
        axis1_idx, axis2_idx = 0, 2
    
    # Calculate unit vectors for the quiver plot
    unit_comp1 = np.zeros_like(comp1)
    unit_comp2 = np.zeros_like(comp2)
    
    # Avoid division by zero
    mag = np.sqrt(comp1**2 + comp2**2)
    mask = mag > 1e-10
    
    unit_comp1[mask] = comp1[mask] / mag[mask]
    unit_comp2[mask] = comp2[mask] / mag[mask]
    
    # Create a quiver plot with unit vectors
    skip = 1  # Adjust to change vector density
    quiv = ax.quiver(grid1[::skip, ::skip], grid2[::skip, ::skip], 
                   unit_comp1[::skip, ::skip], unit_comp2[::skip, ::skip],
                   E_mag[::skip, ::skip],  # Color by magnitude
                   cmap='viridis',
                   scale=20,  # Adjusted for vector size
                   width=0.004,  # Width of arrow
                   headwidth=3,  # Head width
                   pivot='mid')  # Arrow pivots at midpoint
    
    # Add colorbar for the field magnitude
    cbar = plt.colorbar(quiv, ax=ax)
    cbar.set_label('Electric Field Magnitude (N/C)')
    
    # Draw the plates as lines
    if plate_orientation == 'xy' and slice_plane == 'xz':
        # For xy plates viewed in xz slice, draw the edges of the plates as lines
        plate1_z = plate1_center[2]
        plate2_z = plate2_center[2]
        x_min = -plate_width/2
        x_max = plate_width/2
        
        # Top plate edge (positive)
        ax.plot([x_min, x_max], [plate1_z, plate1_z], 'r-', linewidth=3, label='Positive Plate (+)')
        # Add + symbols along the plate
        num_symbols = 10
        for i in range(num_symbols):
            x_pos = x_min + (x_max - x_min) * (i + 0.5) / num_symbols
            ax.text(x_pos, plate1_z, '+', color='red', fontsize=14, 
                   ha='center', va='center', fontweight='bold')
        
        # Bottom plate edge (negative)
        ax.plot([x_min, x_max], [plate2_z, plate2_z], 'b-', linewidth=3, label='Negative Plate (-)')
        # Add - symbols along the plate
        for i in range(num_symbols):
            x_pos = x_min + (x_max - x_min) * (i + 0.5) / num_symbols
            ax.text(x_pos, plate2_z, '−', color='blue', fontsize=14, 
                   ha='center', va='center', fontweight='bold')
        
        # Draw the dielectric as a thick colored line with different colors for surfaces
        dielectric_z = dielectric_center[2]
        dielectric_top = dielectric_z + dielectric_thickness/2
        dielectric_bottom = dielectric_z - dielectric_thickness/2
        
        # Fill the dielectric region
        ax.fill_between([x_min, x_max], dielectric_bottom, dielectric_top, 
                       color='green', alpha=0.3, label='Dielectric')
        
        # Extract only the Z coordinates from positions to get surface positions at this slice
        if len(pos_surface_positions) > 0 and len(neg_surface_positions) > 0:
            pos_z = [pos[2] for pos in pos_surface_positions]
            neg_z = [pos[2] for pos in neg_surface_positions]
        
            # Draw the top edge of the dielectric (positive surface charge)
            ax.plot([x_min, x_max], [dielectric_top, dielectric_top], 'r-', linewidth=2, alpha=0.8)
            
            # Draw the bottom edge of the dielectric (negative surface charge)
            ax.plot([x_min, x_max], [dielectric_bottom, dielectric_bottom], 'b-', linewidth=2, alpha=0.8)
        
            # Label the dielectric with polarization charges
            num_diel_symbols = 10
            for i in range(num_diel_symbols):
                x_pos = x_min + (x_max - x_min) * (i + 0.5) / num_diel_symbols
                # Add δ+ and δ- symbols to show polarization
                ax.text(x_pos, dielectric_top, 'δ+', color='darkred', fontsize=12, 
                       ha='center', va='center', fontweight='bold')
                ax.text(x_pos, dielectric_bottom, 'δ-', color='darkblue', fontsize=12, 
                       ha='center', va='center', fontweight='bold')
    
    # Add gridlines
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Set labels and title
    ax.set_xlabel(f'{ax1_label} (m)')
    ax.set_ylabel(f'{ax2_label} (m)')
    ax.set_title(f'Electric Field in Capacitor with Dielectric: {slice_plane.upper()}-Plane View')
    
    # Set aspect ratio
    ax.set_aspect('equal')
    
    # Add legend
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    
    return fig

def generate_capacitor_with_dielectric(width, height, nx, ny, plate_separation, 
                                     dielectric_position=0.0,  # 0.0 means centered
                                     dielectric_thickness=0.05,  # thickness of dielectric
                                     plate_orientation='xy', 
                                     center_position=[0, 0, 0],
                                     plate_charge_density=1e-6,
                                     dielectric_surface_charge_density=1e-7):
    """
    Generate a capacitor with a dielectric slab inside.
    """
    # Generate the capacitor plates first
    plate_charges, plate_positions, plate1_center, plate2_center, normal_axis = generate_parallel_plates(
        width, height, nx, ny,
        plate_separation=plate_separation,
        plate_orientation=plate_orientation,
        center_position=center_position,
        charge_density=plate_charge_density
    )
    
    # Calculate dielectric position based on the relative position parameter
    # dielectric_position = 0 means centered, 1 means at positive plate, -1 means at negative plate
    if plate_orientation == 'xy':
        dielectric_z = center_position[2] + (plate_separation/2) * dielectric_position
        dielectric_center = np.array([center_position[0], center_position[1], dielectric_z])
    elif plate_orientation == 'yz':
        dielectric_x = center_position[0] + (plate_separation/2) * dielectric_position
        dielectric_center = np.array([dielectric_x, center_position[1], center_position[2]])
    elif plate_orientation == 'xz':
        dielectric_y = center_position[1] + (plate_separation/2) * dielectric_position
        dielectric_center = np.array([center_position[0], dielectric_y, center_position[2]])
    
    # Generate the dielectric
    dielectric_charges, dielectric_positions, actual_thickness, pos_positions, neg_positions = generate_dielectric_plate(
        width, height, nx, ny,
        plate_orientation=plate_orientation,
        center_position=dielectric_center,
        surface_charge_density=dielectric_surface_charge_density,
        thickness=dielectric_thickness
    )
    
    # Combine all charges and positions
    all_charges = np.concatenate((plate_charges, dielectric_charges))
    all_positions = np.concatenate((plate_positions, dielectric_positions))
    
    return (all_charges, all_positions, plate1_center, plate2_center, 
            dielectric_center, actual_thickness, normal_axis, pos_positions, neg_positions)

def main():
    """
    Main function to run the capacitor simulation with dielectric.
    """
    # Configuration
    plate_width = 1.0  # meters
    plate_height = 1.0  # meters
    plate_separation = 0.5  # meters
    
    # Number of charges to represent each plate
    nx = 20  # points along width
    ny = 20  # points along height
    
    # Charge densities
    plate_charge_density = 1e-6  # C/m² for plates
    dielectric_surface_charge_density = 8e-7  # C/m² for dielectric surfaces - increased for better visibility
    
    # Dielectric thickness
    dielectric_thickness = 0.1  # meters
    
    # Dielectric position (0 = centered, 1 = at positive plate, -1 = at negative plate)
    dielectric_position = 0.2  # slightly offset from center toward positive plate for better visualization
    
    # Orientation of the capacitor plates
    plate_orientation = 'xy'
    
    # Generate the capacitor with dielectric
    (charges, positions, plate1_center, plate2_center, dielectric_center, 
     actual_thickness, normal_axis, pos_positions, neg_positions) = generate_capacitor_with_dielectric(
        plate_width, plate_height, nx, ny, 
        plate_separation=plate_separation,
        dielectric_position=dielectric_position,
        dielectric_thickness=dielectric_thickness,
        plate_orientation=plate_orientation,
        center_position=[0, 0, 0],
        plate_charge_density=plate_charge_density,
        dielectric_surface_charge_density=dielectric_surface_charge_density
    )
    
    print(f"Generated capacitor with {len(charges)} charges")
    print(f"Plate orientation: {plate_orientation}-plane")
    print(f"Plate separation: {plate_separation} m")
    print(f"Plate 1 center (positive): {plate1_center}")
    print(f"Plate 2 center (negative): {plate2_center}")
    print(f"Dielectric center: {dielectric_center}")
    print(f"Dielectric thickness: {actual_thickness} m")
    
    # For xy-oriented plates, choose xz plane to see the cross-section
    if plate_orientation == 'xy':
        slice_plane = 'xz'  # slice along x-z plane to see the field between plates
    elif plate_orientation == 'yz':
        slice_plane = 'xz'  # slice along x-z plane
    elif plate_orientation == 'xz':
        slice_plane = 'xy'  # slice along x-y plane
        
    # Visualize the cross-section with unit vectors
    fig = visualize_cross_section_vectors(
        charges, positions, 
        plate1_center, plate2_center, 
        dielectric_center, actual_thickness, 
        normal_axis,
        plate_width, plate_height, 
        plate_orientation, 
        slice_plane,
        pos_positions, neg_positions
    )
    
    plt.savefig('capacitor_with_dielectric_focused.png', dpi=300)
    plt.show()

if __name__ == "__main__":
    main()

Generated capacitor with 1600 charges
Plate orientation: xy-plane
Plate separation: 0.5 m
Plate 1 center (positive): [0.   0.   0.25]
Plate 2 center (negative): [ 0.    0.   -0.25]
Dielectric center: [0.   0.   0.05]
Dielectric thickness: 0.1 m


NameError: name 'plate_separation' is not defined